In [1]:
from tensorflow import keras
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers, losses, metrics
from tensorflow.keras import preprocessing

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
import os
import re

from konlpy.tag import Okt

import matplotlib.pyplot as plt

In [2]:
base_dir = 'C:/Users/user/Desktop/인공지능 개발자 양성과정/산학프로젝트'  # 경로

excel_file = 'C:/Users/user/Desktop/인공지능 개발자 양성과정/산학프로젝트/네이버뉴스크롤링7월1주차.xlsx'      # 파일명 

In [3]:
excel_dir = os.path.join(base_dir, excel_file)

excel_dir

'C:/Users/user/Desktop/인공지능 개발자 양성과정/산학프로젝트/네이버뉴스크롤링7월1주차.xlsx'

In [4]:
df=pd.read_excel(excel_dir, header=0)

x=df['title']
x

0                        쏟아지는 신작 게임… 뜨거운 모바일 大戰
1                넥슨, ‘바람의나라: 연’ 7월15일 국내 정식 서비스
2                 코로나19에도 지스타 강행…"눈치싸움 중인 게임업계"
3                    넥슨, '바람의 나라 : 연' 15일 출시 확정
4      엔씨소프트, 리니지2M 돌풍 글로벌로...하반기 해외사업 강화에 '방점'
                         ...                   
589     PS5 독점 타이틀 '리틀 데빌 인사이드', 이렇게 주목받을 줄 몰랐다
590          ‘코로나19’ 펜데믹발 경기침체 후 50대 부자들 희비 엇갈려
591                    [업앤다운]게임주 상승…액션스퀘어↑·미투온↓
592              넥슨, 바람의나라 ‘빽투더바람’ 업데이트 사전예약 실시
593                       넥슨 '던전앤파이터' 등 여름맞이 공세
Name: title, Length: 594, dtype: object

In [5]:
# 태그 단어
PAD = "<PADDING>"   # 패딩
OOV = "<OOV>"       # 없는 단어(Out of Vocabulary)

# 태그 인덱스
PAD_INDEX = 0
OOV_INDEX = 1

# 데이터 타입
#ENCODER_INPUT  = 0
#DECODER_INPUT  = 1
#DECODER_TARGET = 2

# 한 문장에서 단어 시퀀스의 최대 개수 
max_sequences = 100
max_words=100
# 임베딩 벡터 차원
embedding_dim = 100

# LSTM 히든레이어 차원
lstm_hidden_dim = 128

# 정규 표현식 필터
RE_FILTER = re.compile("=[.,!?\"':;~()]")


In [6]:
# 형태소분석 함수
def pos_tag(sentences):
    
    # KoNLPy 형태소분석기 설정
    tagger = Okt()
    
    # 문장 품사 변수 초기화
    sentences_pos = []
    
    # 모든 문장 반복
    for sentence in sentences:
        # 특수기호 제거
        sentence = re.sub(RE_FILTER, "", str(sentence))
        
        # 배열인 형태소분석의 출력을 띄어쓰기로 구분하여 붙임
        sentence = " ".join(tagger.morphs(sentence))
        sentences_pos.append(sentence)
        
    return sentences_pos

# 기사 제목 마이닝

In [7]:
x_new = pos_tag(x)

sentences = []
sentences.extend(x_new)


words = []

# 단어들의 배열 생성
for sentence in sentences:
    for word in sentence.split():
        words.append(word)

# 길이가 0인 단어는 삭제
words = [word for word in words if len(word) > 0]

# 중복된 단어 삭제
words = list(set(words))

# 제일 앞에 태그 단어 삽입
words[:0] = [PAD, OOV]

#print(words) # words : test,trainset의 모든 단어들이 들어간 list

# 단어와 인덱스의 단어사전 생성
word_to_index = {word: index for index, word in enumerate(words)}  # 단어 : 인덱스값 (인코딩시 사용)  # 단어 : 인덱스값
index_to_word = {index: word for index, word in enumerate(words)}  # 인덱스값 : 단어 (디코딩시 사용)  # 인덱스값 : 단어

x_new


['쏟아지는 신작 게임 … 뜨거운 모바일 大戰',
 '넥슨 , ‘ 바람의나라 : 연 ’ 7월 15일 국내 정식 서비스',
 '코로나 19 에도 지스타 강행 …" 눈치 싸움 중인 게임 업계 "',
 "넥슨 , ' 바람 의 나라 : 연 ' 15일 출시 확정",
 "엔씨소프트 , 리니지 2 M 돌풍 글로벌 로 ... 하반기 해외 사업 강화 에 ' 방점 '",
 '대규모 업데이트 통해 새로운 이야기 시작 하는 게임 사 들',
 "넥슨 , ' 서든어택 ' 여름 대규모 캠페인 ' 찐 업데이트 ' 예고",
 '[ 뉴스 브리핑 ] 7월 3일 … 엔씨 vs 넥슨 신작 대전 · SK 바이오 팜 상장 첫날 · 카카오 ...',
 '넷게임즈 , 넥슨 과 글로벌 시장 진출 기대 - 현대차',
 '[ IT 이슈 리 마인드 ] ③ 남궁훈 카카오 게임 즈 대표 “ 우리 의 진짜 경쟁자 는 ‘ 나 ...',
 'IT 업계 , 코로나 재 확산 조짐 에 재택근무 상 시체 계 전환',
 '상반기 게임 업계 , 코로나 19 여파 에도 성과',
 '컴투스 , + 1.30% 52 주 신고 가',
 '상반기 이용자 눈길 사로잡은 게임 10 선',
 '옴니텔 , + 0.63% 52 주 신고 가',
 'SBI 인베스트먼트 , + 7.86% 상승 폭 확대',
 "[ 리포트 브리핑 ] 넷게임즈 , ' 넥슨 과 함께 글로벌 진출 ' Not Rated - 현대차 증권",
 "[ 여의도 클라 쓰 ] ' 바람 의 나라 ' 모바일 버전 이 끌어 올릴 ' 넥슨 지 티 '",
 'SBI 인베스트먼트 , + 16.53% VI 발동',
 '넷마블 , + 1.89% 52 주 신고 가',
 '엠게임 , + 3.79% 52 주 신고 가',
 "넥슨 , 기 대작 ' 바람의나라 : 연 ' 7월 15일 국내 정식 서비스",
 '엔씨 · 넷마블 잇는 대어 나오나 - 카카오 게임 즈 상장 재도전 … 몸값 2조 예상',
 "' 코로나 19 무풍지대 ' 빅 3 게임 사 , 2분 기 실적 도 기대",
 "7월 모바일 기 대작 쏟아진다 …' 대형

In [8]:
import operator

lst=[]
for i in range(len(x_new)):
    w=x_new[i]
    w=w.split(' ')
    for j in range(len(w)):
        lst.append(w[j])

d={}
ex_t='[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…...》'

for i in lst:
    if i in ex_t:
        continue
    if i not in d:
        d[i]=1
    else:
        d[i]+=1
        
d=sorted(d.items(), key = lambda item: item[1])
d2={}

for i in range(len(d)):
    d2[d[i][0]]=d[i][1]
    

for k,v in d2.items():
    print('{} : {}'.format(k,v))


쏟아지는 : 1
뜨거운 : 1
지스타 : 1
강행 : 1
눈치 : 1
싸움 : 1
중인 : 1
돌풍 : 1
새로운 : 1
이야기 : 1
첫날 : 1
마인드 : 1
남궁훈 : 1
우리 : 1
진짜 : 1
경쟁자 : 1
재 : 1
확산 : 1
재택근무 : 1
시체 : 1
전환 : 1
여파 : 1
성과 : 1
1.30% : 1
사로잡은 : 1
0.63% : 1
7.86% : 1
리포트 : 1
Not : 1
Rated : 1
증권 : 1
여의도 : 1
클라 : 1
쓰 : 1
끌어 : 1
올릴 : 1
16.53% : 1
1.89% : 1
잇는 : 1
대어 : 1
나오나 : 1
2조 : 1
예상 : 1
무풍지대 : 1
쏟아진다 : 1
챕 : 1
터 : 1
마지막 : 1
흔드 : 1
핑크 : 1
조합 : 1
숙옷 : 1
모델 : 1
걸그룹 : 1
이클립스 : 1
집 : 1
즐겨요 : 1
빛나는 : 1
에린 : 1
콘서트 : 1
쏜다 : 1
도약 : 1
정비 : 1
10.22% : 1
없이 : 1
황금알 : 1
찾자 : 1
12일 : 1
킹스 : 1
베스파 : 1
룽투 : 1
코리아 : 1
2.09% : 1
V : 1
군단 : 1
장 : 1
힐라 : 1
11.30% : 1
13.19% : 1
띄운 : 1
레전드 : 1
제노 : 1
뉴 : 1
노멀 : 1
방역 : 1
낳은 : 1
변화 : 1
추첨 : 1
싸인 : 1
폴라 : 1
로이 : 1
5.51% : 1
발급 : 1
좋은 : 1
사람 : 1
대거 : 1
주년 : 1
기념 : 1
9.38% : 1
삼 : 1
담다 : 1
경기장 : 1
짓는다 : 1
7000 : 1
평 : 1
규모 : 1
2023년 : 1
말 : 1
완공 : 1
'··· : 1
펼친다 : 1
개인 : 1
7.99% : 1
6 : 1
선행 : 1
온다 : 1
아프리카 : 1
BJ : 1
들을 : 1
매혹 : 1
시킨 : 1
비결 : 1
100만 : 1
돌파 : 1
치열해진 : 1
사용 : 1
굳히기 : 1
정무 : 1
감각 : 1
없다 : 1
윤석열 : 1
열

In [9]:
res=pd.DataFrame(list(d2.items()),
                   columns=['단어', '빈도수'])
res.to_excel('넥슨_텍스트마이닝_제목만2.xlsx')